In [75]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [76]:
attendance = pd.read_csv('attendance.csv',parse_dates=True)
holiday = pd.read_csv('holidays.csv',parse_dates=True)
attendance.head()

C:\Users\Jathushan\AppData\Local\Temp\ipykernel_4092\2576389047.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  attendance = pd.read_csv('attendance.csv',parse_dates=True)


,id,project_code,date,out_date,Employee_No,in_time,out_time,Hourly_Time,Shift_Start,Shift_End
0,280220,100.0,2/1/2021,2/1/2021,1272,8:30:00,17:00:00,8.5,8:30:00,17:00:00
1,280307,1.0,2/1/2021,2/1/2021,1539,8:30:00,17:00:00,8.5,8:30:00,17:00:00
2,280312,1.0,2/1/2021,2/1/2021,1540,7:45:00,17:00:00,9.25,8:30:00,17:00:00
3,280385,100.0,2/1/2021,2/1/2021,1273,8:00:00,18:00:00,10,8:30:00,17:00:00
4,322526,1.0,2/1/2021,2/1/2021,573,23:22:28,23:22:28,0,8:30:00,17:00:00


In [77]:
# extend two ndarrays
holiday1 = np.append(holiday.columns.values,holiday.iloc[:,0].values)

#create a df with the extended ndarrays
holiday2 = pd.DataFrame(holiday1)
holiday.columns = ['h_date']
holiday.head()


,h_date
0,2017-05-14
1,2017-05-21
2,2017-05-28
3,2017-06-04
4,2017-06-11


In [78]:
# Check if 'date' and 'out_date' columns are equal
isequal = attendance['date'].equals(attendance['out_date'])

# unequal count
unequal_count = sum(attendance['date'] != attendance['out_date'])
print(unequal_count)


10111


In [79]:
# Convert time columns to datetime format
attendance['in_time'] = pd.to_datetime(attendance['in_time'], format='%H:%M:%S', errors='coerce').dt.time
attendance['out_time'] = pd.to_datetime(attendance['out_time'], format='%H:%M:%S', errors='coerce').dt.time
attendance['Shift_Start'] = pd.to_datetime(attendance['Shift_Start'], format='%H:%M:%S', errors='coerce').dt.time
attendance['Shift_End'] = pd.to_datetime(attendance['Shift_End'], format='%H:%M:%S', errors='coerce').dt.time

In [80]:
attendance['onTimeIN'] = (attendance['in_time'] <= attendance['Shift_Start']).astype(int)

# Create 'OUT' column to indicate if left after shift end time
attendance['onTimeOUT'] = (attendance['out_time'] >= attendance['Shift_End']).astype(int)

In [81]:
attendance.head()

,id,project_code,date,out_date,Employee_No,in_time,out_time,Hourly_Time,Shift_Start,Shift_End,onTimeIN,onTimeOUT
0,280220,100.0,2/1/2021,2/1/2021,1272,08:30:00,17:00:00,8.5,08:30:00,17:00:00,1,1
1,280307,1.0,2/1/2021,2/1/2021,1539,08:30:00,17:00:00,8.5,08:30:00,17:00:00,1,1
2,280312,1.0,2/1/2021,2/1/2021,1540,07:45:00,17:00:00,9.25,08:30:00,17:00:00,1,1
3,280385,100.0,2/1/2021,2/1/2021,1273,08:00:00,18:00:00,10,08:30:00,17:00:00,1,1
4,322526,1.0,2/1/2021,2/1/2021,573,23:22:28,23:22:28,0,08:30:00,17:00:00,0,1


In [82]:
# selct only onTimeIN and onTimeOUT columns, Employee_No
attendance = attendance[['Employee_No','onTimeIN','onTimeOUT']]
attendance.head()

,Employee_No,onTimeIN,onTimeOUT
0,1272,1,1
1,1539,1,1
2,1540,1,1
3,1273,1,1
4,573,0,1


In [84]:
#group by Employee_No and sum onTimeIN and onTimeOUT
attendance = attendance.groupby(['Employee_No']).sum().reset_index()
attendance.head()

,Employee_No,onTimeIN,onTimeOUT
0,347,121,457
1,348,460,524
2,349,6,63
3,351,111,94
4,352,399,550


In [85]:
attendance.to_csv('attendance_preprocessed.csv',index=False)